# Install Libraries and Packages

In [ ]:
! pip install streamlit -q

In [ ]:
pip install stqdm

In [ ]:
! pip install openai

# Stable Diffusion XL Packages

In [ ]:
pip install diffusers --upgrade

In [ ]:
pip install invisible_watermark transformers accelerate safetensors

# Creating app.py file inside colab

In [ ]:
%%writefile app.py
import time
import openai
import torch
import streamlit as st
from stqdm import stqdm
from diffusers import DiffusionPipeline

#Stable Diffusion XL setup
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0",
                                         torch_dtype=torch.float16,
                                         use_safetensors=True,
                                         variant="fp16")
pipe.to("cuda")

st.set_page_config(
    page_title="StoryCraft",
    page_icon="👾",
)

with st.sidebar:
  with st.form("API KEY"):
    openai_api_key = st.text_input("OpenAI API key", value="", type="password")
    st.caption("*If you don't have an OpenAI API key, get it [here](https://platform.openai.com/account/api-keys).*")
    submitted_key = st.form_submit_button('Submit')



if submitted_key:
  try:
      openai.api_key = openai_api_key
      response = openai.Completion.create(engine="davinci", prompt="test")
  except Exception as e:
      st.warning(f'Invalid OpenAI API key: {e}')


#%%
#GPT-3.5-Paramters
#(A) Story Generator
story_gen = """
Generate a short and engaging storyline for a popular video game.
The storyline should be concise, yet captivating, and provide an overview of the
game's plot, characters, and setting. The game can be from any genre, such as action,
adventure, role-playing, or simulation
"""

story_user1 = """
Can you give me short story line of one of the triple games in the market.
"""

story_response1 = """
The game is set in a post-apocalyptic world where a deadly virus has mutated
most of the world's population and destroyed the world as we know it.
The story follows Joel, who is tasked with escorting the young Ellie
across a post-apocalyptic United States and defend against cannibalistic creatures
infected by a mutated strain of the Cordyceps fungus2.
Ellie is being smuggled out of the city for reasons unknown
"""

story_user2 = """
Can me a summary of God of War Ragnarok storyline?
"""

story_response2 = """
Continuing the story, God of War Ragnarok begins in Midgard place three years
after the previous game. As Kratos and Atreus travel home from training, Freya,
who still holds a grudge against Kratos for killing her son, attacks them.
They manage to fend her off and make it back inside the protection stave.
Back at their home, Atreus finds the dying Fenrir and, during the wolf's last breaths,
unintentionally transfers its soul to his knife. Atreus buries Fenrir where he
loses control of his emotions and transforms into a bear named Bjorn.
Kratos subdues Bjorn, and Atreus returns to his human form—to both of their surprises.
Kratos goes back home with Atreus to call it a day, but only a few moments later,
Thor visits their home—exactly how Atreus saw it in his dreams years ago.
Odin follows him inside, offering a truce if Kratos agrees to stop spilling Aesir
blood and if Atreus stops searching for Tyr.Kratos turns down the deal and
fights Thor, who knocked him to Tyr's Temple with Mjolnir. The fight ends in a
draw and Kratos returns home quickly with Brok and Sindri's help.
"""

story_user3 = """
can give me a short story line about chill farming simulation games such as
stardew valley?
"""

story_response3 = """
Stardew Valley is a farming simulation game that begins with you inheriting
your grandfather's old farm plot in Stardew Valley. You must learn to live off
the land and create a new life for yourself1. After character creation, you are
given an intro video that tells a more in-depth story about you, your grandfather,
and the land you inherit. As the scene begins, it appears that you are in the
room of your grandfather. You are standing in front of your grandfather who
lay sick in bed. He presents you with an envelope as you two talk; however, he
tells you not to open it. After you take the envelope your grandfather tells
you that there will be a day where you will feel crushed by the burden of
life and your spirit will fade. When that day comes that is when you are to open
the envelope. After his speech to you, he asks for rest and the screen fades to black.
In the game, you learn that your grandfather passed away
"""

#(B) Translator
ts_system = """
You are the best translator and you can translate from one language and give
output to another language.
"""

ts_input1 = """
hello
"""

ts_response1 = """
안녕하세요
"""

#(C) Image Description Generator
idg_system = """
You are a game designer and experience in generate short detailed image
description for a popular video game based on its short storyline.
The description should be concise, yet informative, and provide an overview of
the game's visual elements, such as characters, setting, and atmosphere.
The game can be from any genre, such as action, adventure, role-playing, or
simulation. The output must be in one sentence
"""
idg_input1 = """
give me a short clear iamge description about the game
"""

idg_response1 = """
the game would likely feature a vast and immersive open world with desolate
cities, overgrown forests, and abandoned settlements.
"""



  #%%

def story_ai(text_input):
    response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role":"system",
            "content": story_gen
        },

        {
            "role":"user",
            "content" : story_user1
        },

        {
            "role":"assistant",
            "content": story_response1
        },

        {
            "role":"user",
            "content" : story_user2
        },

        {
            "role":"assistant",
            "content": story_response2
        },

        {
            "role":"user",
            "content" : story_user3
        },

        {
            "role":"assistant",
            "content": story_response3
        },

        {
            "role":"user",
            "content" : text_input
        }
        ],
    max_tokens = 1000,
    temperature = 1
    )

    storyline = response['choices'][0]['message']['content']
    return storyline

def translator(storyline,lang_input):
    translator_result = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role":"system",
            "content": ts_system
        },

        {
            "role":"user",
            "content" : ts_input1
        },

        {
            "role":"assistant",
            "content": ts_response1
        },

        {
            "role":"user",
            "content" : f"Translate this story line: {storyline} into {lang_input}"
        }
    ],
    max_tokens = 1000,
    temperature = 1
    )

    translated = translator_result['choices'][0]['message']['content']
    return translated

def img_desc(storyline):

    response_img = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role":"system",
            "content": idg_system
        },

        {
            "role":"user",
            "content" : idg_input1
        },

        {
            "role":"assistant",
            "content": idg_response1
        },

        {
    "role":"user",
    "content" : f"""
    Generate a short clear image description in one sentence about how
    the game will looks based on the story: {storyline}.
    The output must be compact in on sentence only.
    """
    }
    ],
    max_tokens = 1000,
    temperature = 1
    )

    img_desc = response_img['choices'][0]['message']['content']
    return img_desc

def stable_diffusion_xl(img_desc):
    prompt = f"""
    Generate an visual stunning video game images based on the description:
    {img_desc}
                      """
    images = pipe(prompt=prompt).images[0]
    return images

#%%
# Setup Streamlit App
# Define custom styles for justified text
justified_text_style = '''
<style>
.justified-text {
    text-align: justify;
}
</style>
'''
st.markdown(justified_text_style, unsafe_allow_html=True)

info = """
This app uses OpenAI's GPT-3.5-Turbo model to generate captivating
and immersive storylines for your video games. It also allows you to generate an
visual stunning images by using Stable Diffusion XL. Plus, You can choose the
story line output in several languages
"""


col1, col2 = st.columns([2,3])

col1.markdown("# StoryCraft 👾")
col1.markdown(f'<div class="justified-text">{info}</div>', unsafe_allow_html=True)

form = col2.form

with form('input_form'):
      text_input = st.text_area("Input your idea here: ")
      lang_input = st.selectbox("Choose language output", ("English",
                                                            "Spanish",
                                                            "Deutsch",
                                                            "French",
                                                            "Korean",
                                                            "Japanese"))
      submitted = st.form_submit_button('Submit')

      text_inputs = [text_input]

if submitted:


    for i in stqdm(range(100), backend=True, frontend=True):
      time.sleep(0.5)

    story = story_ai(text_input)
    if lang_input == "English":
        st.markdown('**_The Story:_** ')
        st.write(story)
    else:
        translated_story = translator(story,lang_input)
        st.markdown(f'**_The Story in {lang_input}:_** ')
        st.write(translated_story)
    image_desc = img_desc(story)
    st.markdown('**_The Image Description:_** ')
    st.write(image_desc)
    for i in stqdm(range(100), backend=True, frontend=True):
      time.sleep(0.5)
    st.markdown('**_Image Generated Using Stable Diffusion XL:_** ')
    image = stable_diffusion_xl(image_desc)
    st.image(image)

# To Generate Your Public IP

In [ ]:
print("############")
print("IP v")
!curl https://ipv4.icanhazip.com/
print("############")

# Run Streamlit using localtunnel

In [ ]:
!streamlit run app.py & npx localtunnel -p 8501

In [ ]:
import time
import openai
import torch
import streamlit as st
from stqdm import stqdm
from diffusers import DiffusionPipeline

#Stable Diffusion XL setup
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0",
                                         torch_dtype=torch.float16,
                                         use_safetensors=True,
                                         variant="fp16")
pipe.to("cuda")

st.set_page_config(
    page_title="StoryCraft",
    page_icon="👾",
)

with st.sidebar:
  with st.form("API KEY"):
    openai_api_key = st.text_input("OpenAI API key", value="", type="password")
    st.caption("*If you don't have an OpenAI API key, get it [here](https://platform.openai.com/account/api-keys).*")
    submitted_key = st.form_submit_button('Submit')



if submitted_key:
  try:
      openai.api_key = openai_api_key
      response = openai.Completion.create(engine="davinci", prompt="test")
  except Exception as e:
      st.warning(f'Invalid OpenAI API key: {e}')


#%%
#GPT-3.5-Paramters
#(A) Story Generator
story_gen = """
Generate a short and engaging storyline for a popular video game.
The storyline should be concise, yet captivating, and provide an overview of the
game's plot, characters, and setting. The game can be from any genre, such as action,
adventure, role-playing, or simulation
"""

story_user1 = """
Can you give me short story line of one of the triple games in the market.
"""

story_response1 = """
The game is set in a post-apocalyptic world where a deadly virus has mutated
most of the world's population and destroyed the world as we know it.
The story follows Joel, who is tasked with escorting the young Ellie
across a post-apocalyptic United States and defend against cannibalistic creatures
infected by a mutated strain of the Cordyceps fungus2.
Ellie is being smuggled out of the city for reasons unknown
"""

story_user2 = """
Can me a summary of God of War Ragnarok storyline?
"""

story_response2 = """
Continuing the story, God of War Ragnarok begins in Midgard place three years
after the previous game. As Kratos and Atreus travel home from training, Freya,
who still holds a grudge against Kratos for killing her son, attacks them.
They manage to fend her off and make it back inside the protection stave.
Back at their home, Atreus finds the dying Fenrir and, during the wolf's last breaths,
unintentionally transfers its soul to his knife. Atreus buries Fenrir where he
loses control of his emotions and transforms into a bear named Bjorn.
Kratos subdues Bjorn, and Atreus returns to his human form—to both of their surprises.
Kratos goes back home with Atreus to call it a day, but only a few moments later,
Thor visits their home—exactly how Atreus saw it in his dreams years ago.
Odin follows him inside, offering a truce if Kratos agrees to stop spilling Aesir
blood and if Atreus stops searching for Tyr.Kratos turns down the deal and
fights Thor, who knocked him to Tyr's Temple with Mjolnir. The fight ends in a
draw and Kratos returns home quickly with Brok and Sindri's help.
"""

story_user3 = """
can give me a short story line about chill farming simulation games such as
stardew valley?
"""

story_response3 = """
Stardew Valley is a farming simulation game that begins with you inheriting
your grandfather's old farm plot in Stardew Valley. You must learn to live off
the land and create a new life for yourself1. After character creation, you are
given an intro video that tells a more in-depth story about you, your grandfather,
and the land you inherit. As the scene begins, it appears that you are in the
room of your grandfather. You are standing in front of your grandfather who
lay sick in bed. He presents you with an envelope as you two talk; however, he
tells you not to open it. After you take the envelope your grandfather tells
you that there will be a day where you will feel crushed by the burden of
life and your spirit will fade. When that day comes that is when you are to open
the envelope. After his speech to you, he asks for rest and the screen fades to black.
In the game, you learn that your grandfather passed away
"""

#(B) Translator
ts_system = """
You are the best translator and you can translate from one language and give
output to another language.
"""

ts_input1 = """
hello
"""

ts_response1 = """
안녕하세요
"""

#(C) Image Description Generator
idg_system = """
You are a game designer and experience in generate short detailed image
description for a popular video game based on its short storyline.
The description should be concise, yet informative, and provide an overview of
the game's visual elements, such as characters, setting, and atmosphere.
The game can be from any genre, such as action, adventure, role-playing, or
simulation. The output must be in one sentence
"""
idg_input1 = """
give me a short clear iamge description about the game
"""

idg_response1 = """
the game would likely feature a vast and immersive open world with desolate
cities, overgrown forests, and abandoned settlements.
"""



  #%%

def story_ai(text_input):
    response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role":"system",
            "content": story_gen
        },

        {
            "role":"user",
            "content" : story_user1
        },

        {
            "role":"assistant",
            "content": story_response1
        },

        {
            "role":"user",
            "content" : story_user2
        },

        {
            "role":"assistant",
            "content": story_response2
        },

        {
            "role":"user",
            "content" : story_user3
        },

        {
            "role":"assistant",
            "content": story_response3
        },

        {
            "role":"user",
            "content" : text_input
        }
        ],
    max_tokens = 1000,
    temperature = 1
    )

    storyline = response['choices'][0]['message']['content']
    return storyline

def translator(storyline,lang_input):
    translator_result = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role":"system",
            "content": ts_system
        },

        {
            "role":"user",
            "content" : ts_input1
        },

        {
            "role":"assistant",
            "content": ts_response1
        },

        {
            "role":"user",
            "content" : f"Translate this story line: {storyline} into {lang_input}"
        }
    ],
    max_tokens = 1000,
    temperature = 1
    )

    translated = translator_result['choices'][0]['message']['content']
    return translated

def img_desc(storyline):

    response_img = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role":"system",
            "content": idg_system
        },

        {
            "role":"user",
            "content" : idg_input1
        },

        {
            "role":"assistant",
            "content": idg_response1
        },

        {
    "role":"user",
    "content" : f"""
    Generate a short clear image description in one sentence about how
    the game will looks based on the story: {storyline}.
    The output must be compact in on sentence only.
    """
    }
    ],
    max_tokens = 1000,
    temperature = 1
    )

    img_desc = response_img['choices'][0]['message']['content']
    return img_desc

def stable_diffusion_xl(img_desc):
    prompt = f"""
    Generate an visual stunning video game images based on the description:
    {img_desc}
                      """
    images = pipe(prompt=prompt).images[0]
    return images

#%%
# Setup Streamlit App
# Define custom styles for justified text
justified_text_style = '''
<style>
.justified-text {
    text-align: justify;
}
</style>
'''
st.markdown(justified_text_style, unsafe_allow_html=True)

info = """
This app uses OpenAI's GPT-3.5-Turbo model to generate captivating
and immersive storylines for your video games. It also allows you to generate
visual stunning images by using Stable Diffusion XL. Plus, You can choose the
story line output in several languages
"""


col1, col2 = st.columns([2,3])

col1.markdown("# StoryCraft 👾")
col1.markdown(f'<div class="justified-text">{info}</div>', unsafe_allow_html=True)

form = col2.form

with form('input_form'):
      text_input = st.text_area("Input your idea here: ")
      lang_input = st.selectbox("Choose language output", ("English",
                                                            "Spanish",
                                                            "Deutsch",
                                                            "French",
                                                            "Korean",
                                                            "Japanese"))
      submitted = st.form_submit_button('Submit')

      text_inputs = [text_input]

if submitted:


    for i in stqdm(range(100), backend=True, frontend=True):
      time.sleep(0.5)

    story = story_ai(text_input)
    if lang_input == "English":
        st.markdown('**_The Story:_** ')
        st.write(story)
    else:
        translated_story = translator(story,lang_input)
        st.markdown(f'**_The Story in {lang_input}:_** ')
        st.write(translated_story)
    image_desc = img_desc(story)
    st.markdown('**_The Image Description:_** ')
    st.write(image_desc)

    for i in stqdm(range(100), backend=True, frontend=True):
      time.sleep(0.5)

    st.markdown('**_Image Generated Using Stable Diffusion XL:_** ')
    image = stable_diffusion_xl(image_desc)
    st.image(image)

